In [33]:
import os
import sys
import numpy as np
from collections import defaultdict, Counter
import time
from matplotlib import pyplot
import json5
from matplotlib import pyplot as pyplot
from scipy.cluster.hierarchy import dendrogram
import pandas
import logging
logging.basicConfig(format='%(message)s', level=logging.DEBUG)

import importlib
import duhportinf
import duhportinf._optimize
import duhportinf.busdef
import duhportinf.main_portinf as main
from   duhportinf.busdef import BusDef
from   duhportinf._optimize import MatchCost
from   duhportinf import util
from   duhportinf._bundle import BundleTree, Interface

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
block_root = '/ip-block-designs'
bus_spec_rootdir = '/bus-defs/specs'

In [70]:
pcie_json_path = os.path.join(
    block_root,
    'block-plda-pcie-controller',
    'PLDA_ws.json5',
)
ddr_json_path = os.path.join(
    block_root,
    'block-cadence-ddr3',
    'ddr.json5',
)
nvdla_json_path = os.path.join(
    block_root,
    'block-nvdla',
    'nvdla.json5',
)
ark_json_path = os.path.join(
    block_root,
    'block-ark',
    'ark.json5',
)
pcie_ports  = util.get_unassigned_ports(pcie_json_path)
ddr_ports   = util.get_unassigned_ports(ddr_json_path)
nvdla_ports = util.get_unassigned_ports(nvdla_json_path)
ark_ports   = util.get_unassigned_ports(ark_json_path)

In [55]:
bus_defs = duhportinf.load_bus_defs(bus_spec_rootdir)

In [76]:
ddr_axi0_ports = [p for p in ddr_ports if p[0].startswith('axi0')]
#ddr_axi0_ports.append(('bs', 0, 0))
print('{} axi0 ddr ports'.format(len(ddr_axi0_ports)))
ddr_i_bus_mappings = main.get_bus_matches(ddr_axi0_ports, list(bus_defs))

56 axi0 ddr ports


In [78]:
for interface, bus_mappings in ddr_i_bus_mappings:
    print('size:{}'.format(interface.size))
util.dump_json_bus_candidates('testy.json', ddr_json_path, ddr_i_bus_mappings)

size:56


In [ ]:
def debug(ports, dport, bus_abstract_names):
    # filter to get debug bus defs
    bus_abstract_names = set(bus_abstract_names)
    dbus_defs = list(filter(
        lambda bd: bd.abstract_type.name in bus_abstract_names,
        bus_defs,
    ))
    
    pg, Z, wire_names = duhportinf._grouper.get_port_grouper(ports)
    pg_bus_pairings = duhportinf.main._get_bus_pairings(pg, dbus_defs)
    # filter only for port_group with dport
    pg_bus_pairings = list(filter(
        lambda x: dport in x[2],
        pg_bus_pairings,
    ))
    pg_bus_mappings = duhportinf.main._get_initial_bus_matches(pg, pg_bus_pairings)
    return list(map(lambda x: x[2:], pg_bus_mappings))
    

In [ ]:
port_path = '/ip-block-designs/tmp/tmp.json5' 
ports = duhportinf.get_ports_from_json5(port_path)
dport = ('front_port_axi4_0_ar_ready', 1, -1)
pg_bus_mappings = debug(ports, dport, ['AXI4_rtl'])

In [47]:
stime = time.time()

nvdla_pg_bus_mappings = duhportinf.get_bus_matches(nvdla_ports, list(bus_defs))
etime = time.time()
print('total time: {}s'.format(etime-stime))

total time: 13.745474338531494s                                                  


In [48]:
util.dump_json_bus_candidates('testy.json', nvdla_json_path, nvdla_pg_bus_mappings)

In [16]:
bt = BundleTree(ddr_ports)
ddr_bus_pairings = main._get_bus_pairings(bt, list(bus_defs))
ddr_bus_mappings = main._get_initial_bus_matches(bt, ddr_bus_pairings)

In [17]:
print('pairings')
for nid, _, interface, _ in ddr_bus_pairings:
    print('  - interface size:{}, prefix:{}'.format(interface.size, interface.prefix))
print('mappings')
for nid, _, interface, _ in ddr_bus_mappings:
    print('  - interface size:{}, prefix:{}'.format(interface.size, interface.prefix))

pairings
  - interface size:8, prefix:lp_ext_
  - interface size:51, prefix:axi1_
  - interface size:51, prefix:axi0_
  - interface size:5, prefix:phy_reg_
  - interface size:8, prefix:phy_reg
  - interface size:13, prefix:regH
  - interface size:5, prefix:dfi_wrdata_
  - interface size:6, prefix:dfi_rddata_
  - interface size:5, prefix:dfi_rdlvl_
  - interface size:4, prefix:dfi_wrlvl_
  - interface size:51, prefix:dfi_
mappings
  - interface size:13, prefix:regH
  - interface size:5, prefix:phy_reg_
  - interface size:4, prefix:dfi_wrlvl_
  - interface size:5, prefix:dfi_wrdata_
  - interface size:5, prefix:dfi_rdlvl_
  - interface size:6, prefix:dfi_rddata_
  - interface size:8, prefix:lp_ext_
  - interface size:51, prefix:axi0_
  - interface size:51, prefix:axi1_
  - interface size:51, prefix:dfi_


In [6]:
stime = time.time()

ddr_pg_bus_mappings = duhportinf.get_bus_matches(ddr_ports, list(bus_defs))

etime = time.time()
print('total time: {}s'.format(etime-stime))

total time: 36.72839069366455s                                                   


In [8]:
util.dump_json_bus_candidates('testy.json', ddr_json_path, ddr_pg_bus_mappings)

In [46]:
s_pcie_ports = list(filter(lambda p: 'axi4' not in p[0], pcie_ports))
stime = time.time()
pcie_pg_bus_mappings = duhportinf.get_bus_matches(s_pcie_ports, list(bus_defs))

etime = time.time()
print('total time: {}s'.format(etime-stime))

hierarchically clustering ports and selecting port groups
  - done
initial bus pairing with port groups
  - done
bus mapping
  - done                                                                         
total time: 13.784493684768677s


In [51]:
dport = ('pl_pll_ack', 1, 1)
for interface, bus_mappings in pcie_pg_bus_mappings:
    if dport in interface.ports:
        print('port group')
        for port in interface.ports:
            print('  - ', port)
        print('bus mappings')
        for bm in bus_mappings:
            print('  - cost:{} bus_def:{}'.format(
                bm.cost,
                str(bm.bus_def.abstract_type),
            ))

port group
  -  ('pl_rxdata', None, 1)
  -  ('pl_rxstatus', None, 1)
  -  ('pl_phystatus', None, 1)
  -  ('pl_rxstandby', None, -1)
  -  ('pl_rxpolarity', None, -1)
  -  ('pl_txcompliance', None, -1)
  -  ('pl_txelecidle', None, -1)
  -  ('pl_txsyncheader', None, -1)
  -  ('pl_txstartblock', None, -1)
  -  ('pl_txdatavalid', None, -1)
  -  ('pl_txdatak', None, -1)
  -  ('pl_txdata', None, -1)
  -  ('pl_txdetectrx', None, -1)
  -  ('pl_pll_rate', 3, -1)
  -  ('pl_pll_ack', 1, 1)
  -  ('pl_width', 2, -1)
  -  ('pl_ltssm', 5, -1)
  -  ('pl_blockaligncontrol', 1, -1)
  -  ('pl_txdeemph', 1, -1)
  -  ('pl_txswing', 1, -1)
  -  ('pl_txmargin', 3, -1)
  -  ('pl_rate', 2, -1)
  -  ('pl_powerdown', 2, -1)
  -  ('pl_spor', 1, 1)
  -  ('pl_npor', 1, 1)
  -  ('pl_srst', 1, 1)
  -  ('pl_rstnp', 1, 1)
  -  ('pl_rstn', 1, 1)
  -  ('pl_rstn_srst_out', 1, -1)
  -  ('pl_pclk_change_ok', None, 1)
  -  ('pl_pclk_change_ack', None, -1)
  -  ('pl_pclk', 1, 1)
  -  ('pl_pclk_rate', 3, -1)
bus mappings
  - co

In [ ]:
def dump_nonaxi_dendogram(path, ports):
    assert path.endswith('.pdf')
    nonaxi_ports = list(filter(
        lambda p: not p[0].startswith('axi'),
        ports,
    ))
    _, Z, wire_names = abxportinf.get_port_grouper(nonaxi_ports)
    fig, ax = pyplot.subplots(1,1, figsize=(10,40))
    _ = dendrogram(
        Z,
        ax=ax,
        orientation='left',
        labels=wire_names,
    )
    pyplot.savefig(path)


In [ ]:
dump_nonaxi_dendogram('ddr-nonaxi-dendogram.pdf', ddr_ports)
dump_nonaxi_dendogram('pcie-nonaxi-dendogram.pdf', pcie_ports)

In [ ]:
util.dump_json_bus_candidates('ddr-busprop.json', ddr_pg_bus_mappings)
util.dump_json_bus_candidates('pcie-busprop.json', pcie_pg_bus_mappings)